## LangChain Agent기반 포괄적 데이터 탐색

In [1]:

print("환경 설정 시작...")

# 환경 변수에서 API 키 가져오기
import os
import warnings
from dotenv import load_dotenv

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")
else:
    print("OpenAI API 키가 성공적으로 로드되었습니다.")

print("환경 설정이 완료되었습니다.")

환경 설정 시작...
OpenAI API 키가 성공적으로 로드되었습니다.
환경 설정이 완료되었습니다.


In [2]:

print("라이브러리 로드 중...")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rcParams['font.family'] = ['나눔고딕', 'NanumGothic', 'Malgun Gothic']
plt.rcParams['axes.unicode_minus'] = False

# Seaborn 스타일 설정
sns.set_style("whitegrid")
sns.set_palette("husl")

# pandas 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("라이브러리 로드가 완료되었습니다.")

라이브러리 로드 중...
라이브러리 로드가 완료되었습니다.


In [3]:

print("데이터 로드 시작...")

# 데이터 로드
try:
    train = pd.read_csv('./data/titanic/train.csv')
    test = pd.read_csv('./data/titanic/test.csv')
    
    print(f"훈련 데이터 로드 완료: {train.shape}")
    print(f"테스트 데이터 로드 완료: {test.shape}")
    print("데이터 로드가 성공적으로 완료되었습니다.")
except FileNotFoundError:
    print("Titanic 데이터 파일을 찾을 수 없습니다.")
    print("현재 디렉토리에 train.csv와 test.csv 파일을 배치해주세요.")
    
    # 샘플 Titanic 데이터 생성
    print("샘플 Titanic 데이터를 생성합니다...")
    np.random.seed(42)
    n_samples = 891
    
    train = pd.DataFrame({
        'PassengerId': range(1, n_samples + 1),
        'Survived': np.random.choice([0, 1], n_samples, p=[0.62, 0.38]),
        'Pclass': np.random.choice([1, 2, 3], n_samples, p=[0.24, 0.21, 0.55]),
        'Name': [f'Name_{i}' for i in range(1, n_samples + 1)],
        'Sex': np.random.choice(['male', 'female'], n_samples, p=[0.65, 0.35]),
        'Age': np.random.normal(29.7, 14.5, n_samples),
        'SibSp': np.random.choice([0, 1, 2, 3, 4, 5], n_samples, p=[0.68, 0.23, 0.06, 0.02, 0.01, 0.00]),
        'Parch': np.random.choice([0, 1, 2, 3, 4, 5, 6], n_samples, p=[0.76, 0.13, 0.08, 0.02, 0.01, 0.00, 0.00]),
        'Ticket': [f'Ticket_{i}' for i in range(1, n_samples + 1)],
        'Fare': np.random.lognormal(3, 1, n_samples),
        'Cabin': np.random.choice(['A1', 'B2', 'C3', None], n_samples, p=[0.1, 0.1, 0.1, 0.7]),
        'Embarked': np.random.choice(['C', 'Q', 'S'], n_samples, p=[0.19, 0.09, 0.72])
    })
    # 나이 결측값 추가 (약 20%)
    train.loc[train.sample(frac=0.2).index, 'Age'] = np.nan
    
    test = train.drop(['Survived'], axis=1).head(418).copy()
    test['PassengerId'] = range(892, 892 + 418)
    
    print("샘플 데이터 생성이 완료되었습니다.")

print("\n=== 데이터 기본 정보 ===")
print(f"훈련 데이터: {train.shape[0]}행 {train.shape[1]}열")
print(f"테스트 데이터: {test.shape[0]}행 {test.shape[1]}열")
print(f"타겟 변수: {'Survived (생존 여부)' if 'Survived' in train.columns else '없음'}")

데이터 로드 시작...
Titanic 데이터 파일을 찾을 수 없습니다.
현재 디렉토리에 train.csv와 test.csv 파일을 배치해주세요.
샘플 Titanic 데이터를 생성합니다...
샘플 데이터 생성이 완료되었습니다.

=== 데이터 기본 정보 ===
훈련 데이터: 891행 12열
테스트 데이터: 418행 11열
타겟 변수: Survived (생존 여부)
